In [0]:
import sys
import os

# 1. Configura Path para importar 'src'
repo_root = os.getcwd()
repo_root = '/'.join(repo_root.split('/')[:-1])
if repo_root not in sys.path:
    sys.path.append(repo_root)




In [0]:
%pip install -r ../requirements.txt

In [0]:
from src.utility.environment import Environment
from src.infrastructure.data_manager import DataManager
from src.infrastructure.ml_trainer import PySparkTrainer

In [0]:
# 1. Defina um caminho temporário dentro do SEU volume existente
tmp_path = Environment.MLFLOW_DFS_TMP

# 2. Cria a pasta se ela não existir (para evitar erros de caminho inválido)
if not os.path.exists(tmp_path):
    os.makedirs(tmp_path)

# 3. Define a variável de ambiente que o MLflow exige
os.environ['MLFLOW_DFS_TMP'] = tmp_path

In [0]:
# Importa múltiplos algoritmos
from pyspark.ml.classification import RandomForestClassifier, GBTClassifier, LogisticRegression

# 1. Carregar Dados
manager = DataManager(spark)
df_gold = manager.read_delta(f"{Environment.feature_store_path}/churn")

# 2. Instancia o Trainer Genérico
current_user = dbutils.notebook.entry_point.getDbutils().notebook().getContext().userName().get()
experiment_name = "churn_initial_experiment"
full_experiment_path = f"/Users/{current_user}/{experiment_name}"
trainer = PySparkTrainer(full_experiment_path)

In [0]:
import pyspark.sql.functions as F

# 1. Calcule a proporção de cada classe
total = df_gold.count()
count_churn = df_gold.filter(F.col('churn') == 1).count()
count_nao_churn = total - count_churn

# 2. Defina os pesos (Estratégia: Inversamente proporcional à frequência)
# Se Churn é raro, ele ganha peso alto.
peso_churn = total / (2 * count_churn)
peso_nao_churn = total / (2 * count_nao_churn)

print(f"Peso Churn (1): {peso_churn:.2f}")
print(f"Peso Não-Churn (0): {peso_nao_churn:.2f}")



In [0]:
# 3. Adicione a coluna 'weight' ao DataFrame
df_gold = df_gold.withColumn(
    "weight",
    F.when(F.col("churn") == 1, F.lit(peso_churn))
     .otherwise(F.lit(peso_nao_churn))
)

In [0]:
# 4. Definindo as colunas utilizadas pelo modelo
features = ['recency', 'frequency', 'monetary', 'total_items_volume', 'payment_method_count', 'max_installments', 'avg_satisfaction', 'min_review_score', 'total_reviews_given']

In [0]:

# CENÁRIO A: Treinando Random Forest (Baseline)
rf = RandomForestClassifier(
    labelCol="churn", 
    featuresCol="features", 
    weightCol="weight",
    numTrees=50, 
    maxDepth=5,
    seed=42
)

print("🚀 Iniciando Experimento: Random Forest")
rf_run_id = trainer.train(df_gold, estimator=rf, run_name="RF_Baseline", feature_cols=features)


In [0]:

# CENÁRIO B: Treinando Gradient Boosted Trees (GBT)
gbt = GBTClassifier(
    labelCol="churn", 
    featuresCol="features",
    weightCol="weight",
    maxIter=20,
    maxDepth=5,
    seed=42
)

print("\n🚀 Iniciando Experimento: GBT Classifier")
gbt_run_id = trainer.train(df_gold, estimator=gbt, run_name="GBT_Baseline", feature_cols=features)

In [0]:
# CENÁRIO C: Regressão Logística (Mais simples e explicável)
lr = LogisticRegression(
    labelCol="churn",
    featuresCol="features",
    weightCol="weight",
    regParam=0.01,
    elasticNetParam=1
)

print("\n🚀 Iniciando Experimento: Logistic Regression")
lr_run_id = trainer.train(df_gold, estimator=lr, run_name="LogisticReg_Baseline", feature_cols=features)

# Definindo um modelo para produção
Como visto, o modelo deve ser avaliado frente a métricas que capturam perspectivas de desbalanceamento, por isso aqui irei avaliar frente à métrica F1-Score, ficando entre o GBT e o RandomForest. Ambos possuem um Throughput e uso de RAM parecidos, portanto utilizarei o GBT por ter um F1, ligeiramente maior. Por isso, ele foi adicionado como modelo de produção manualmente.